In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jan-2021/train.csv
/kaggle/input/tabular-playground-series-jan-2021/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')


In [3]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')


In [4]:
Xtrain = train.drop(['id','target'], axis = 1)
ytrain = train['target']

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(Xtrain, ytrain, test_size = 0.3, random_state = 2021)

In [6]:
import lightgbm as lgb

lgb4 = lgb.LGBMRegressor(random_state=33, n_estimators=50000, min_data_per_group=5, boosting_type='gbdt', num_leaves=246, max_depth= 24, learning_rate=0.0005, subsample_for_bin=100000,
lambda_l1= 1.074622455507616e-05, lambda_l2= 2.0521330798729704e-06, n_jobs=-1, cat_smooth=1.0, 
 importance_type='split', metric='rmse', min_child_samples=100, min_gain_to_split=0.0, feature_fraction=0.5, 
 bagging_freq=6, min_sum_hessian_in_leaf=0.001, min_data_in_leaf=100, bagging_fraction=0.82063411)

In [7]:
lgb4.fit(X_train, y_train, eval_set = [(X_test, y_test)], eval_metric = 'rmse', early_stopping_rounds = 100, verbose = True)

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] lambda_l1 is set=1.074622455507616e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.074622455507616e-05
[LightGBM] [Warning] bagging_fraction is set=0.82063411, subsample=1.0 will be ignored. Current value: bagging_fraction=0.82063411
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.001, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.001
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=100 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] min_gain_to_split is set=0.0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.0
[LightGBM] [Warning] lambda_l2 is set=2.0521330798729704e-06, reg_lambda=0.0 will be ignored. Current val

LGBMRegressor(bagging_fraction=0.82063411, bagging_freq=6, cat_smooth=1.0,
              feature_fraction=0.5, lambda_l1=1.074622455507616e-05,
              lambda_l2=2.0521330798729704e-06, learning_rate=0.0005,
              max_depth=24, metric='rmse', min_child_samples=100,
              min_data_in_leaf=100, min_data_per_group=5, min_gain_to_split=0.0,
              min_sum_hessian_in_leaf=0.001, n_estimators=50000, num_leaves=246,
              random_state=33, subsample_for_bin=100000)

2697 , 50-> 0.695987
3094, 100 -> 0.695904
25 ->0.695877
22 -> 0.695832
24 ->0.695815

100000 -> 0.695689
80000 -> 0.695684

In [8]:
y_pred = lgb4.predict(test.drop('id', axis = 1), num_iteration= lgb4.best_iteration_ )

In [9]:
output = pd.DataFrame({'id' : test['id'], 'target' : y_pred})
output.to_csv('submission7-v2.csv', index = False)